# Convolutional Neural Network

This file applies conlutional neural networks to the classification problem. In the last part, the code applies simple majority voting from five classifiers.

In [52]:
# Import standard libraries, helper functions and keras
import csv
import numpy as np
import keras
import preprocess
import common

from sklearn import svm
from sklearn.cross_validation import StratifiedKFold

from keras import backend
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import BatchNormalization, Activation, Flatten, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

backend.set_image_dim_ordering('th')

### Data Loading

In [53]:
"""
This cell loads the training and the testing data.
"""

X_train = preprocess.load_data(common.X_train_file, reshape=True)
X_test = preprocess.load_data(common.X_test_file, reshape=True)
y_train = preprocess.load_label(common.y_train_file)

X_train = common.normalize(X_train)
X_test = common.normalize(X_test)

X_train, y_train = preprocess.augment(X_train, y_train)

### Definition of CNN Models

In [54]:
"""
This cell defines all Convolutional Neural Network models. It also defines
a cross validation function for CNN models.
"""

def model_1():
    """
    Defines a basic CNN model with one conv layer, one pooling layer,
    and two dense layers.
    """
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),
                     padding='valid',
                     activation='relu',
                     input_shape=common.INPUT_SHAPE))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='uniform'))
    model.add(Dense(common.NUM_CLASSES, activation='softmax', kernel_initializer='uniform'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def model_2():
    """
    Defines a basic CNN model with two conv layer, two pooling layers,
    and two dense layers.
    """
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),
                     input_shape=common.INPUT_SHAPE,
                     padding='valid',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, kernel_size=(5, 5),
                     padding='valid',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(common.NUM_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def model_3():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5),
                     input_shape=common.INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())

    model.add(Conv2D(32, kernel_size=(5, 5)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D())

    model.add(Conv2D(32, kernel_size=(5, 5)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D())

    model.add(Flatten(name='flatten'))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(common.NUM_CLASSES))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=0.01),
                  metrics=['accuracy'])
    return model


def cross_val(model, X, y):
    skf = StratifiedKFold(y, n_folds=common.N_FOLDS, shuffle=True)
    y = keras.utils.to_categorical(y, common.NUM_CLASSES)
    scores = []

    for i, (train, test) in enumerate(skf):
        print ("Running Fold: ", i+1, "/", common.N_FOLDS)

        model.fit(X[train], y[train],
                  batch_size=common.BATCH_SIZE,
                  epochs=40,
                  verbose=1)

        score = model.evaluate(X[test], y[test], verbose=0)
        scores.append(score[0])
    
    print("Cross validaion score: ", np.mean(scores))

### Application of CNN Models

In [55]:
"""
This cell applies the CNN models.
"""

y_train = keras.utils.to_categorical(y_train, common.NUM_CLASSES)

# Train three CNN models
model1 = model_1()
model1.fit(X_train, y_train,
           batch_size=64,
           epochs=40,
           verbose=1)

model2 = model_2()
model2.fit(X_train, y_train,
           batch_size=64,
           epochs=40,
           verbose=1)

model3 = model_3()
model3.fit(X_train, y_train,
           batch_size=128,
           epochs=150,
           verbose=1)

# Predict results for test set and save to file
y_pred = model3.predict_classes(X_test)
common.save_results('submission_CNN_more_epochs.csv', y_pred)

Epoch 1/40
1932/1932 [==============================] - 4s 2ms/step - loss: 1.6838 - acc: 0.3986
Epoch 2/40
1932/1932 [==============================] - 4s 2ms/step - loss: 1.5377 - acc: 0.4524
Epoch 3/40
1932/1932 [==============================] - 5s 2ms/step - loss: 1.3386 - acc: 0.5347
Epoch 4/40
1932/1932 [==============================] - 4s 2ms/step - loss: 1.1200 - acc: 0.6056
Epoch 5/40
1932/1932 [==============================] - 4s 2ms/step - loss: 0.8977 - acc: 0.6837
Epoch 6/40
1932/1932 [==============================] - 4s 2ms/step - loss: 0.7516 - acc: 0.7552
Epoch 7/40
1932/1932 [==============================] - 4s 2ms/step - loss: 0.6005 - acc: 0.8157
Epoch 8/40
1932/1932 [==============================] - 4s 2ms/step - loss: 0.5217 - acc: 0.8390
Epoch 9/40
1932/1932 [==============================] - 4s 2ms/step - loss: 0.4408 - acc: 0.8737
Epoch 10/40
1932/1932 [==============================] - 4s 2ms/step - loss: 0.3895 - acc: 0.8820
Epoch 11/40
1932/1932 [======

1932/1932 [==============================] - 18s 10ms/step - loss: 1.7129 - acc: 0.4027
Epoch 5/150
1932/1932 [==============================] - 19s 10ms/step - loss: 1.6932 - acc: 0.4037
Epoch 6/150
1932/1932 [==============================] - 18s 9ms/step - loss: 1.6904 - acc: 0.4094
Epoch 7/150
1932/1932 [==============================] - 18s 9ms/step - loss: 1.6847 - acc: 0.4110
Epoch 8/150
1932/1932 [==============================] - 18s 10ms/step - loss: 1.6608 - acc: 0.4120
Epoch 9/150
1932/1932 [==============================] - 17s 9ms/step - loss: 1.6562 - acc: 0.4125
Epoch 10/150
1932/1932 [==============================] - 18s 10ms/step - loss: 1.6434 - acc: 0.4151
Epoch 11/150
1932/1932 [==============================] - 18s 9ms/step - loss: 1.6277 - acc: 0.4208
Epoch 12/150
1932/1932 [==============================] - 18s 9ms/step - loss: 1.6054 - acc: 0.4312
Epoch 13/150
1932/1932 [==============================] - 19s 10ms/step - loss: 1.5953 - acc: 0.4286
Epoch 14/150


1932/1932 [==============================] - 17s 9ms/step - loss: 0.3562 - acc: 0.8929
Epoch 86/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.2956 - acc: 0.9136
Epoch 87/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.2913 - acc: 0.9099
Epoch 88/150
1932/1932 [==============================] - 19s 10ms/step - loss: 0.3138 - acc: 0.9027
Epoch 89/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.2580 - acc: 0.9255
Epoch 90/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.2574 - acc: 0.9234
Epoch 91/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.2176 - acc: 0.9441
Epoch 92/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.3764 - acc: 0.8778
Epoch 93/150
1932/1932 [==============================] - 17s 9ms/step - loss: 0.2325 - acc: 0.9322
Epoch 94/150
1932/1932 [==============================] - 20s 10ms/step - loss: 0.2164 - acc: 0.9436
Epoch 95/15

## Train SVM Models for Simple Majority Voting

In [58]:
# Prepares data for SVM use
X_train_svm = preprocess.load_data(common.X_train_file)
y_train_svm = preprocess.load_label(common.y_train_file)
X_test_svm = preprocess.load_data(common.X_test_file)

# Prepares two SVM models
model4 = svm.SVC(kernel='linear')
model4.fit(X_train_svm, y_train_svm)

model5 = svm.SVC(kernel='rbf', degree=2)
model5.fit(X_train_svm, y_train_svm)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Application of Simple Majority Voting

In [68]:
def majority_vote(predictions):
    """
    Given a list of predictions, return the majority vote of these predictions.
    """
    result = []
    n = len(predictions[0])

    for i in range(n):
        i_preds = []
        for p in predictions:
            i_preds.append(p[i])

        median = int(np.median(i_preds))
        result.append(median)

    return result

"""
Simple majority voting.
"""

models = [model1, model2, model3, model4, model5]
predictions = []
for m in models:
    try:
        # CNN
        p = m.predict_classes(X_test)
    except:
        # SVM
        p = m.predict(X_test_svm)

    predictions.append(p)

result = majority_vote(predictions)
common.save_results('majority.csv', result)

322/322 [==============================] - 1s 4ms/step


In [ ]:
# Please use model3 as the final model for prediction
model3 = model_3()
model3.fit(X_train, y_train,
           batch_size=128,
           epochs=150,
           verbose=1)